# 🔹 Команды Linux (установка python, venv, docker, jupyter)

``` Ctrl + Alt + T ``` - открыть терминал

```pwd ```  - показывает, где ты сейчас

``ls``  - показывает, что лежит в папке

``` cd folder_name``` - зайти в папку

```cd ..``` - выйти на уровень выше

```mkdir test``` - создать папку

```rm file.txt``` - удалить файл

```rm -r folder``` - удалить папку

## 🔹 УСТАНОВКА PYTHON

```sudo apt update```

```sudo apt upgrade -y```

```sudo apt install python3 python3-pip -y```

```python3 --version```

Должно быть что-то вроде:
```Python 3.10.x```

Проверяем pip:
```pip3 --version```

Проверка: Python работает
```python3```


Если увидим:
```>>>```

Python запустился

Выйти:
```exit()```

## 🔹 УСТАНОВКА JUPYTER NOTEBOOK

```pip3 install --user notebook jupyterlab```

Запуск: ```jupyter notebook```

## 🔹 УСТАНОВКА VS CODE (ОПЦИОНАЛЬНО)

```sudo snap install code --classic```

Запуск:
```code```

## 🔹 УСТАНОВКА ВИРТУАЛЬНОГО ОКРУЖЕНИЯ (VENV)

Создаём папку проекта

```cd ~```

```mkdir hackathon_ml```

```cd hackathon_ml```


ВСЁ ДЕЛАЕМ ТОЛЬКО ТУТ

Создаём venv
```python3 -m venv venv```


АКТИВАЦИЯ venv
```source venv/bin/activate```

## 🔹 УСТАНОВКА БИБЛИОТЕК 
```pip3 install --user numpy pandas scikit-learn xgboost matplotlib```

# 🔹 Пример решения задания Rea2025 с датасетом taxi_trip_pricing.csv (файл задания: C3_M5.pdf) 

Trip_Distance_km          -> число ✅

Time_of_Day               -> текст ❌

Day_of_Week               -> текст ❌

Passenger_Count           -> число ✅

Traffic_Conditions        -> текст ❌

Weather                   -> текст ❌

Base_Fare                 -> число ✅

Per_Km_Rate               -> число ✅

Per_Minute_Rate           -> число ✅

Trip_Duration_Minutes     -> число ✅

Trip_Price                -> число (TARGET)


In [54]:
import pandas as pd

df = pd.read_csv(
    "taxi_trip_pricing.csv",
    na_values=["", " ", "NA", "N/A", "null", "None"]
)

df = df.dropna(subset=["Trip_Price"]) #очистим строки с пропусками в таргете
df
#переведм текстовые категории в числа



,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180
5,8.64,Afternoon,Weekend,2.0,Medium,Clear,2.55,1.71,0.48,89.33,60.2028
...,...,...,...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,Weekend,4.0,Medium,Clear,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,Weekday,4.0,Medium,Clear,3.12,0.61,NaN,61.96,62.1295
997,7.70,Morning,Weekday,3.0,Low,Rain,2.08,1.78,NaN,54.18,33.1236
998,47.56,Morning,Weekday,1.0,Low,Clear,2.67,0.82,0.17,114.94,61.2090


## тут мы будем использовать One-Hot Encoding, чтобы превести текстовые категории в числовые, пропуски можно заполнять не медианой и самым частым значением, а с помощью корреляций, но на шаблоне мы это делать не будем. Об этом будет написано в 2 модуле.

In [55]:
import numpy as np
import pandas as pd

# 1) Числовые колонки: NaN -> медиана
num_cols = df.select_dtypes(include=[np.number]).columns 
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# 2) Категориальные (object): NaN -> мода (самое частое)
obj_cols = df.select_dtypes(include=["object"]).columns  
for c in obj_cols:
    mode = df[c].mode(dropna=True)
    fill_value = mode.iloc[0] if len(mode) else "missing"
    df[c] = df[c].fillna(fill_value)

# 3) object -> числа через one-hot 
df = pd.get_dummies(df, columns=obj_cols, drop_first=False)


In [56]:
df

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,Time_of_Day_Afternoon,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Day_of_Week_Weekday,Day_of_Week_Weekend,Traffic_Conditions_High,Traffic_Conditions_Low,Traffic_Conditions_Medium,Weather_Clear,Weather_Rain,Weather_Snow
0,19.35,3.0,3.56,0.80,0.32,53.82,36.2624,False,False,True,False,True,False,False,True,False,True,False,False
2,36.87,1.0,2.70,1.21,0.15,37.27,52.9032,False,True,False,False,False,True,True,False,False,True,False,False
3,30.33,4.0,3.48,0.51,0.15,116.81,36.4698,False,True,False,False,True,False,False,True,False,True,False,False
4,25.87,3.0,2.93,0.63,0.32,22.64,15.6180,False,True,False,False,True,False,True,False,False,True,False,False
5,8.64,2.0,2.55,1.71,0.48,89.33,60.2028,True,False,False,False,False,True,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5.49,4.0,2.39,0.62,0.49,58.39,34.4049,True,False,False,False,False,True,False,False,True,True,False,False
996,45.95,4.0,3.12,0.61,0.29,61.96,62.1295,False,False,False,True,True,False,False,False,True,True,False,False
997,7.70,3.0,2.08,1.78,0.29,54.18,33.1236,False,False,True,False,True,False,False,True,False,False,True,False
998,47.56,1.0,2.67,0.82,0.17,114.94,61.2090,False,False,True,False,True,False,False,True,False,True,False,False


In [57]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Trip_Price"]) # дклим на обучающую и тестовые выборки
y = df["Trip_Price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


## Дальше выполним эти пункты:

2. Испытано как минимум три алгоритма разной сущности (RandomForest и XGBoost – разные
сущности, а XGBoost и GradientBoosting – нет). Для оценки точности следует использовать
тестовую выборку
3. Испытанные алгоритмы сравнены по точности, а также по скорости работы

Мы берём 3 разные сущности:

1) LinearRegression — линейная модель

2) RandomForestRegressor — bagging / ансамбль деревьев

3) XGBRegressor — boosting (градиентный бустинг деревьев)

In [58]:
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score


from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ),
    "XGBoost": XGBRegressor(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        random_state=42,
        n_jobs=-1,
        objective="reg:squarederror"
    )
}


In [59]:
results = {}
for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    acc = accuracy_score(
            np.round(y_test),
            np.round(y_pred)
        )
    results[name] = {
        "R2": r2,
        "RMSE": rmse,
        "Accuracy": acc,
        "Train_time_sec": train_time
    }

results_df = pd.DataFrame(results).T
results_df

,R2,RMSE,Accuracy,Train_time_sec
LinearRegression,0.876138,17.015053,0.031414,0.003135
RandomForest,0.942329,11.610329,0.047120,0.189356
XGBoost,0.913125,14.249880,0.094241,0.115503


относительно цен r2 и rmse - допустимые

Для регрессии приоритет такой:

1) R² — МАКСИМАЛЬНЫЙ

2) RMSE — МИНИМАЛЬНЫЙ

4) Лучший алгоритм реализован в виде пайплайна, принимающего на вход пути к файлам, а на
выходе возвращает обученную модель, а также метрики модели на валидационной выборке в
виде словаря (необходимые метрики: средний коэффициент детерминации R2 и средний RMSE).

In [60]:
def train_pipeline(df):
    """
    Простой пайплайн обучения RandomForest.
    
    Вход:
    - df — готовый датасет (все признаки числовые, без пропусков)

    Выход:
    - обученная модель
    - словарь метрик (R2, RMSE)
    """

    # 1. Разделяем признаки и цель
    X = df.drop(columns=["Trip_Price"])
    y = df["Trip_Price"]

    # 2. Делим на train и validation
    X_train, X_val, y_train, y_val = train_test_split(
        X,
        y,
        test_size=0.25,
        random_state=42
    )

    # 3. Создаём модель
    model = RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    )

    # 4. Обучаем модель
    model.fit(X_train, y_train)

    # 5. Предсказания
    y_pred = model.predict(X_val)

    # 6. Метрики
    r2 = r2_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    metrics = {
        "R2": r2,
        "RMSE": rmse
    }

    return model, metrics


In [61]:
model, metrics = train_pipeline(df)

metrics


{'R2': 0.9433908089799228, 'RMSE': 11.123012354242325}

# 🔹 Установка Docker 

Выходим из venv:

```deactivate```


Установка:

```sudo apt install docker.io -y```

```sudo systemctl start docker```

```sudo systemctl enable docker```


Проверка:

```sudo docker run hello-world```




ОБЩАЯ ЛОГИКА ЗАДАНИЯ

ЛОКАЛЬНО (на своей машине)
Тут:

 - Jupyter Notebook

 - анализ

 - предобработка

 - отчёт (HTML / PDF)

УДАЛЁННЫЙ СЕРВЕР
Тут:

 - обучение модели

 - Docker

 - train.py

 - model.pkl

 - Metrics.txt


## 1.1 Переходим на Рабочий стол

Открываем терминал (`Ctrl + Alt + T`) и пишешь:

```bash
cd ~/Рабочий\ стол
```

Проверка:

```bash
pwd
```

Должно быть примерно так:

```
/home/c[X]/Рабочий стол
```

---

## 1.2 Создаём папку отчёта

```bash
mkdir C[X]_M5
```


Заходим в неё:

```bash
cd C[X]_M5
```

---

#  ЭТАП 2. JUPYTER NOTEBOOK 

## 2.1 Запуск Jupyter

В терминале:

```bash
jupyter notebook
```

Откроется браузер.

---

## 2.2 Создание ноутбука

В браузере:

1. Нажимаем **New**
2. Выбераем **Python 3**
3. Сразу **СОХРАНЯЕМ**
4. Копируем туда исходный dataset для написания кода в ноутбуке

Название:

```
C[X]_M5.ipynb
```

---

## 2.3 Что должно быть внутри ноутбука

В ноутбуке должны быть **ячейки**:

1. Загрузка датасета
2. Анализ пропусков
3. Кодирование категорий
4. Заполнение пропусков
5. Обучение 3 моделей
6. Сравнение по метрикам
7. Выбор лучшей модели

---

## 2.4 Экспорт в HTML / PDF 

В Jupyter:

* `File`
* `Save and Export Notebook As`
* `HTML`

Сохраняем в папку C[X]_M5 как:

```
C[X]_M5.html
```

---

#  ЭТАП 3. ДАННЫЕ

## 3.1 Где хранить датасет ЛОКАЛЬНО

В папке `C[X]_M5`:

```bash
mkdir data
```

Скопируем туда CSV-файл, например:

```
data/trips.csv
```

---

#  ЭТАП 4. УДАЛЁННЫЙ СЕРВЕР 

## 4.1 Подключение

Активируем docker и затем пишем 

```bash
ssh c[X]@10.0.10.[Y]
```

---

## 4.2 Создание папки BIG_DATA 

```bash
mkdir BIG_DATA
```

```bash
cd BIG_DATA
```

---

## 4.3 Кладём туда данные

Вариант простой (scp с локалки):

На **локальном компе**:

```bash
scp trips.csv c[X]@10.0.10.[Y]:/home/c[X]/BIG_DATA/data.csv
```

На сервере проверяем:

```bash
ls BIG_DATA
```

---

#  ЭТАП 5. PYTHON ФАЙЛ 

## 5.1 На сервере создаём `train.py`

```bash
nano train.py
```

Вставляем наш код:

```python
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error


def train_pipeline(df):
    X = df.drop(columns=["Trip_Price"])
    y = df["Trip_Price"]

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=42
    )

    model = RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    r2 = r2_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    metrics = {
        "R2": r2,
        "RMSE": rmse
    }

    return model, metrics


if __name__ == "__main__":
    df = pd.read_csv("/app/data/data.csv")

    model, metrics = train_pipeline(df)

    with open("model.pkl", "wb") as f:
        pickle.dump(model, f)

    with open("metrics.txt", "w") as f:
        f.write(f"Run time: {datetime.now()}\n")
        for k, v in metrics.items():
            f.write(f"{k}: {v}\n")
```

Сохраняем:

* Ctrl + O
* Enter
* Ctrl + X

---

#  ЭТАП 6. Dockerfile

```bash
nano Dockerfile
```

```Dockerfile
FROM python:3.10-slim

WORKDIR /app

COPY train.py .

RUN pip install pandas numpy scikit-learn

CMD ["python", "train.py"]
```

Сохраняем.

---

#  ЭТАП 7. .dockerignore 

```bash
nano .dockerignore
```

```text
BIG_DATA/
```

---

#  ЭТАП 8. СБОРКА И ЗАПУСК

```bash
sudo docker build -t c[X]_ml .
```

```bash
sudo docker run \
-v /home/c[X]/BIG_DATA:/app/data \
c[X]_ml
```

---

#  ЭТАП 9. ПОЛУЧАЕМ РЕЗУЛЬТАТЫ

На сервере появятся:

```
model.pkl
metrics.txt
```

Скопировать на локалку:

```bash
scp c[X]@10.0.10.[Y]:/home/c[X]/model.pkl ~/Рабочий\ стол/C[X]_M5/
scp c[X]@10.0.10.[Y]:/home/c[X]/metrics.txt ~/Рабочий\ стол/C[X]_M5/
```

---

#  ФИНАЛЬНАЯ ПРОВЕРКА (ЧЕК-ЛИСТ)

В папке
`/home/c[X]/Рабочий стол/C[X]_M5`
ДОЛЖНО БЫТЬ:

✅ `C[X]_M5.ipynb`

✅ `C[X]_M5.html`

✅ `train.py`

✅ `Dockerfile`

✅ `.dockerignore`

✅ `metrics.txt`

✅ `model.pkl`

✅ папка `data/`

---


